In [10]:
# 二级模型单条测试
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import tensorflow.keras.backend as k
import tensorflow.keras as keras
from hanlp.transform.tsv import TSVTaggingTransform
from model_main import build_model, load_vocabs, build
from hanlp.components.pos import RNNPartOfSpeechTagger

In [28]:

save_dir = "save_dir/"
model_dir = "model_dir/0129_best_model/"

def load_model(model_dir):
    transform = TSVTaggingTransform()
    # 读取字典
    load_vocabs(transform, save_dir)
    # # 构建模型
    # model=build(transform)
    # # 加载参数
    # model.load_weights(model_dir + "model.h5")
    # return model, transform
    return transform


def predict(pre_str_list, transform, model):
    pre_chars = []
    for s in pre_str_list:
        pre_chars.append(list(s))
    # pre_chars = [list(pre_str)]
    data, flat = transform.input_to_inputs(pre_chars)
    batch_size=512
    dataset = transform.inputs_to_dataset(data, batch_size=batch_size)
    pre_tags = []
    for idx, batch in enumerate(dataset):
        X = batch[0]
        Y = model.predict(batch)
        for output in transform.Y_to_outputs(Y, X=X, inputs=data[idx*batch_size:]):
            pre_tags.append(output)
    return pre_tags

In [12]:
transform = load_model(model_dir)
new_model = tf.keras.models.load_model(model_dir,compile=False)

In [44]:
pre_str = [list("阿西亚向北一公里甘肃驰辰吉利4s")]
output = predict(pre_str, transform, new_model)
# temp = tagger.predict(pre_str, batch_size=512)
print(output)

[['X', 'X', 'X', 'X', 'X', 'X', 'X', 'R5', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'R25']]


In [23]:
def Process_tag(addr, tag):
    strList = []
    tagCur = ''
    for i in range(len(tag)):
        if tag[i] != 'X' and tag[i] != 'M':
            if i != 0:
                strList.append(',')
                strList.append(tagCur)
                strList.append(',')
            strList.append(addr[i])
            tagCur = tag[i]
        else:
            strList.append(addr[i])
    strList.append(',')
    strList.append(tagCur)
    return ''.join(strList)

def Process_tag_end(addr, tag):
    strList = []
    tagCur = ''
    for i in range(len(tag)):
        if tag[i] != 'X' and tag[i] != 'M':
            strList.append(addr[i])
            tagCur = tag[i]
            if i != 0:
                strList.append(',')
                strList.append(tagCur)
                strList.append(',')
        else:
            strList.append(addr[i])
    return ''.join(strList[:-1])

def process(address):
    ad = address.split(',')
    newAd = []
    for i in range(len(ad)):
        if i%2==0:
            newAd.append(ad[i])
    return "".join(newAd)

def predict(pre_str_list):
    pre_chars = []
    for s in pre_str_list:
        pre_chars.append(list(s))
#     pre_chars = [list(pre_str)]
    data, flat = transform.input_to_inputs(pre_chars)
    dataset = transform.inputs_to_dataset(data, batch_size=512)
    for idx, batch in enumerate(dataset):
        X = batch[0]
        Y = model.predict_on_batch(batch)
        for output in transform.Y_to_outputs(Y, X=X, inputs=data):
            yield output

In [25]:
from hanlp.components.pos import RNNPartOfSpeechTagger
tagger = RNNPartOfSpeechTagger()
tagger.model = new_model
tagger.transform = transform
def pre_all_sequence():
    import pandas as pd
    all_excute_sequence = []
    all_sequence = []
    pre_result = []
    is_same = []
    all_pre_tags = []
    with open("农行1224.txt", "r", encoding="utf-8")as f:
        for line in f.readlines():
            line = line.strip()
            excute_sequence = process(line)
            all_sequence.append(line)
            all_excute_sequence.append(list(excute_sequence))
    
    print("处理完毕，开始预测")
    temp = tagger.predict(all_excute_sequence, batch_size=512)
    print(len(temp))
    for i, pre_tags in enumerate(temp):
        excute_sequence = all_excute_sequence[i]
        sequence = all_sequence[i]
#         res = Process_tag(excute_sequence, pre_tags)
        res = Process_tag_end(excute_sequence, pre_tags)
        print(pre_tags)
        if sequence == res:
            is_same.append(1)
            print(res + " " + str(1))
        else:
            is_same.append(0)
            print(res + " " + str(0))
        pre_result.append(res)
        all_pre_tags.append(pre_tags)
        print(i)
        
    result_df = pd.DataFrame({"all_sequence":all_sequence, "pre_result": pre_result, "is_same": is_same, "all_pre_tags": all_pre_tags})
    result_df.to_excel("pre_result.xlsx")

In [26]:
pre_all_sequence()

处理完毕，开始预测
1377
['X', 'X', 'X', 'X', 'X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'R3', 'X', 'X', 'X', 'X', 'X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'R3', 'X', 'X', 'X', 'X', 'X', 'X', 'R24', 'X', 'X', 'R5', 'X', 'X', 'R6']
广西壮族自治区,R1,南宁市,R2,横县,R3,广西壮族自治区,R1,南宁市,R2,横县,R3,校椅镇罗村村委,R24,庙局屯,R5,59号,R6 0
0
['X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'X', 'R3', 'X', 'X', 'R2', 'X', 'X', 'R3', 'X', 'X', 'R4', 'X', 'X', 'X', 'R24', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'R25']
江苏省,R1,常州市,R2,钟楼区,R3,常州市,R2,钟楼区,R3,邹区镇,R4,龙潭村委,R24,凌家塘市场蔬菜区办公室,R25 0
1
['X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'X', 'R3', 'X', 'X', 'R3', 'X', 'X', 'R4', 'X', 'X', 'X', 'X', 'R24', 'X', 'X', 'R5', 'X', 'X', 'R6']
云南省,R1,玉溪市,R2,澄江县,R3,澄江县,R3,右所镇,R4,旧城村委会,R24,旧城村,R5,10号,R6 0
2
['X', 'X', 'R1', 'X', 'X', 'R3', 'X', 'X', 'R4', 'X', 'R5', 'X', 'X', 'X', 'R30', 'X', 'X', 'X', 'X', 'X', 'X', 'R7']
北京市,R1,太兴区,R3,黄村镇,R4,孙村,R5,新园小区,R30,8-3-101,R7 0
3
['X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'X', 'R3', 'X', 'X', 'R4', 'X', 'X', 'R5', 'X', 'X', '

['X', 'X', 'R1', 'X', 'X', 'X', 'X', 'R2', 'X', 'X', 'R3', 'X', 'X', 'X', 'R21']
内蒙古,R1,呼和浩特市,R2,新城区,R3,自由职业,R21 0
899
['X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'R3', 'X', 'X', 'X', 'X', 'R4']
山西省,R1,临汾市,R2,襄汾,R3,自由职业者,R4 0
900
['X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'X', 'X', 'R3', 'X', 'X', 'X', 'X', 'X', 'R4']
江西省,R1,吉安市,R2,井冈山市,R3,户自由职业者,R4 0
901
['X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'X', 'R3', 'X', 'X', 'X', 'X', 'X', 'R4']
陕西省,R1,安康市,R2,紫阳县,R3,户自由职业者,R4 0
902
['X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'X', 'X', 'R3', 'X', 'X', 'X', 'X', 'X', 'R4']
四川省,R1,泸州市,R2,龙马潭区,R3,户自由职业者,R4 0
903
['X', 'X', 'X', 'X', 'X', 'R4']
户自由职业者,R4 0
904
['X', 'X', 'R1', 'X', 'X', 'R1', 'X', 'R3', 'X', 'R1', 'X', 'R3', 'X', 'X', 'X', 'R4', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'R25']
北京市,R1,北京市,R1,延庆,R3,北京,R1,延庆,R3,八达岭镇,R4,水关长城飞跃影院,R25 0
905
['X', 'X', 'R1', 'X', 'X', 'R3', 'X', 'X', 'X', 'R25', 'X', 'X', 'R7', 'X', 'X', 'X', 'R5', 'X', 'X', 'R6']
北京市,R1,大兴区,R3,大兴机场,R25,3号门,R7,货邮一路,R5,10号,R6 0
906
['X', 'X', 'R1', '

In [6]:
def evaluate_f1(data,model,transform):
    label = ['R1','R2','R3','R4','R5','R6','R7','R20','R21','R22','R23','R24','R25','R30','R31','R90','R99']
    f1_dict = {item: 0 for item in label}
    la_idx = transform.y_to_idx(tf.constant(label)).numpy()
    # print(la_idx)
    pnum_dict = {item: 1e-10 for item in la_idx}
    rnum_dict = {item: 1e-10 for item in la_idx}
    pall_dict = {item: 1e-10 for item in la_idx}
    rall_dict = {item: 1e-10 for item in la_idx}
    viterbi_sequence = []
    true_label = []
    for x,y in tqdm(data):
        # print(tf.shape(x)[0])
        d = x.numpy()
        temp = model.predict(d, batch_size=512)
        temp = tf.argmax(temp, axis=2)
        viterbi_sequence.extend([i for i in temp.numpy()])
        true_label.extend(y.numpy())
        # print(viterbi_sequence)
    for y,pre_sequence in zip(true_label,viterbi_sequence):
        # print(y)
        # acc.update_state(y, viterbi_sequence, tf.sequence_mask(sequence_length, y.shape[1]))
        for pred,true in zip(pre_sequence,y):
            for i in range(len(true)):
                if true[i] in la_idx:
                    if true[i]==pred[i]:
                        right = True
                        j = i
                        while(j<len(true) and true[j] not in la_idx):
                            if pred[j] in la_idx:
                                right = False
                            j += 1
                        if pred[j] not in la_idx:
                            right =False
                        rnum_dict[true[i]] += right
                    rall_dict[true[i]] += 1
                if pred[i] in la_idx:
                    if true[i] == pred[i]:
                        right = True
                        j = i
                        while(j<len(pred) and pred[j] not in la_idx):
                            if true[j] in la_idx:
                                right = False
                            j += 1
                        if true[j] not in la_idx:
                            right =False
                        pnum_dict[true[i]] += right
                    pall_dict[pred[i]] += 1
    for k in f1_dict.keys():
        idx = transform.y_to_idx(tf.constant([k])).numpy()[0]
        p = pnum_dict[idx]/pall_dict[idx]
        r = rnum_dict[idx]/rall_dict[idx]
        f1_dict[k] = [p, r, 2*(p*r)/(p+r)]
    print('f1 score: {}'.format(f1_dict))
    return f1_dict

In [7]:
from tqdm import tqdm

dev_path = '/home/bangsun/research/fank/best_model_code/model_reconstruction/dataset/data0129_bidata/dev0129_all.txt'
batch_size = 512
dev_data = transform.file_to_dataset(dev_path, batch_size=batch_size, shuffle=True, repeat=None)
f1_dict = evaluate_f1(dev_data,new_model,transform)

672it [02:03,  4.09it/s]


TypeError: object of type 'numpy.int64' has no len()

In [8]:
f1 score: {'R1': [0.9990156483427443, 0.9988512512816617, 0.9989334430483907], 'R2': [0.9979684290915023, 0.9986829852828243, 0.9983255793254477],
           'R3': [0.9968688624370164, 0.9985135555562539, 0.997690531177829], 
           'R4': [0.9877977057779427, 0.9848546479379183, 0.9863239814408228], 'R5': [0.9901019911558647, 0.9911152701693882, 0.9906083715455726], 
           'R6': [0.9920340030911902, 0.9912526447467993, 0.9916431700025492],
           'R7': [0.9902816497746281, 0.9876376392633255, 0.9889578773104707], 'R20': [0.9853167795772396, 0.9916600825212888, 0.9884782545284836],
           'R21': [0.9848398647970282, 0.979431538307928, 0.9821282560448547], 
           'R22': [0.9906536456773112, 0.9671853065044457, 0.9787788202563118], 'R23': [0.9929295309922225, 0.9936990363232024, 0.9933141346267199],
           'R24': [0.9923373911673167, 0.9874938743506811, 0.9899097080987611], 
           'R25': [0.9794944342035695, 0.9774970053048885, 0.9784947004032689], 'R30': [0.9748363696145824, 0.9800324356709742, 0.9774274970380673],
           'R31': [0.9840193454557972, 0.9783949825020749, 0.9811991041471807], 
           'R90': [0.9924456396880955, 0.992351764769472, 0.9923987000087838], 'R99': [0.9985107227958697, 0.9810750170715054, 0.9897160852236383]}

SyntaxError: invalid syntax (<ipython-input-8-1e24d05bd2c3>, line 1)

In [ ]:
f1 score: {'R1': [0.9989537963600963, 0.999329671823767, 0.9991416987410001], 'R2': [0.9978486501297845, 0.9996030995329929, 0.9987251043264919], 
           'R3': [0.9968834654359386, 0.9990604840863601, 0.9979707875008044], 'R4': [0.9864598472636947, 0.9870910763958666, 0.9867753608822418], 
           'R5': [0.9900175749380684, 0.9908519313754974, 0.9904345774383263], 'R6': [0.9919957934096751, 0.991236829895561, 0.9916161664287021], 
           'R7': [0.9890318019367853, 0.9867149836606595, 0.9878720344143981], 'R20': [0.9895697899511716, 0.9910157549803967, 0.9902922446383264], 
           'R21': [0.9837786901797133, 0.9835138954309939, 0.9836462749848577], 'R22': [0.9916450929630501, 0.9932229359420119, 0.9924333873104667],
           'R23': [0.9933647042892447, 0.9943745975803993, 0.993869394391004], 'R24': [0.9894269318449808, 0.9884362422695593, 0.9889313389445978],
           'R25': [0.9782247435677038, 0.9766157982808679, 0.9774196087974063], 'R30': [0.9726075216377992, 0.9778280122627195, 0.9752107804280464],
           'R31': [0.9823712290190068, 0.9772709748721332, 0.9798144648685794], 'R90': [0.9920677746600475, 0.9922150114008932, 0.9921413875678775], 
           'R99': [0.9979690905488409, 0.9812487823884669, 0.9895383104125738]}